In [3]:
from lark import InlineTransformer
from lark import Lark

In [166]:
grammar = r"""

    ?start : pipe
    
    ?atom : SIGNED_NUMBER -> number
    
    ?expr : expr SUM_OP term -> bin_op
          | term
    
    ?term : term MULT_OP atom -> bin_op
          | atom
        
    ?pipe : pipe "|>" expr
          | pipe "|>" MULT_OP atom
          | expr
        
    
    SUM_OP : /[+-]/
    MULT_OP : /[*\/]/
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

"""

In [172]:
class PipelineTransformer(InlineTransformer):
    number = float
        
    def pipe(self, op, expr_left, expr_right):  
        return ('+', expr_left, expr_right)
    
    def bin_op(self, left, op, right):
        return (str(op), left, right)
    

In [173]:
pipeline_parser = Lark(grammar, parser="lalr")
pipeline = pipeline_parser.parse
transformer = PipelineTransformer()

In [174]:
res = pipeline("1+1 |> 2*2 |> *3")

In [175]:
print(res.pretty())

pipe
  pipe
    bin_op
      number	1
      +
      number	1
    bin_op
      number	2
      *
      number	2
  *
  number	3



In [176]:
t = transformer.transform(res)

TypeError: pipe() missing 1 required positional argument: 'expr_right'

In [160]:
t

('+', ('+', ('+', 1.0, 1.0), ('*', 2.0, 2.0)), ('*', 3.0))

In [147]:
def eval_expr(expr):
    if isinstance(expr, float):
        return expr
    
    head, *args = expr

    if isinstance(head, tuple):
        return eval_expr(head) + eval_expr(args)
    elif head == '+':
        x, y = args
        return eval_expr(x) + eval_expr(y)
    elif head == '-':
        x, y = args
        return eval_expr(x) - eval_expr(y)
    elif head == '*':
        x, y = args
        return eval_expr(x) * eval_expr(y)
    elif head == '/':
        x, y = args
        return eval_expr(x) / eval_expr(y)

    else:
        raise ValueError('argumento inválido para S-expression: %r' % head)

In [148]:
eval_expr(t)

[2.0, 2.0]
[3.0]


ValueError: not enough values to unpack (expected 2, got 1)

In [39]:
grammar = r"""

start : expr

?expr : func
      | atom
      | pipe

func : SYMBOL "(" (atom ("," atom)*)? ")"

pipe : func "|>" expr

?atom : NUMBER
      | STRING
      | SYMBOL

SYMBOL : /[-+!@$\/\\*%^&~<>|=\w]+/
%import common.SIGNED_NUMBER -> NUMBER
%import common.ESCAPED_STRING -> STRING

%ignore /\s/
"""

In [40]:
t = Lark(grammar)

In [41]:
print(t.parse('x() |> sum(1)').pretty())

start
  pipe
    func	x
    func
      sum
      1

